## ` Send a request to the website of Taiwanindex to get the HTML of the website.`

In [ ]:
import requests
from bs4 import BeautifulSoup
import os

#### The URL of the page to scrape must be changed every time.

In [ ]:
# # The URL of the page to scrape (must be changed every time)
# page_url = "https://taiwanindex.com.tw/news/324"

# # Send a GET request to the page
# response = requests.get(page_url)
# response.raise_for_status()  # Check if the request was successful

# # Parse the HTML content of the page
# soup = BeautifulSoup(response.text, 'html.parser')

### `Step1.Find the URL of the file which you want to download from the source code of the web page.`
> By the way, the ETF ticker of the adjusted constituents are also printed.

In [ ]:
# # Use soup.find_all to find the elements
# # Navigate to the specific table and row

# tables = soup.find_all('table')
# target_numbers = []  # Use a list to preserve the order

# for table in tables:
#     rows = table.find_all('tr')
#     for row in rows:
#         cells = row.find_all('td')
#         for cell in cells:
#             # Find the deepest span elements to avoid duplicates
#             deepest_spans = cell.select('span span span span')

#             # ETF tickers of the adjusted constituents are also listed.
#             for span in deepest_spans:
#                 # Check if the span text is a digit and add to target numbers
#                 if span.text.isdigit():
#                     target_numbers.append(span.text)

# if target_numbers:
#     print("Found numbers:", target_numbers)
# else:
#     print("No numbers found in the specified path")

### `Step2.Download the file URL of the HTML you found.`
> On Taiwanindex's website, the location of the HTML URLs of downloadable files are all standardized, they all start with "https://backend.taiwanindex.com.tw/api/downloadFile/TechnicalNotices/" and end with a specific number.
>
>If you find the pattern of the URL, the download will be faster and the time will be reduced dramatically.

In [ ]:
# # Find the specific link to download the file
# # You can use more specific criteria to find the correct link

# download_link_tags = soup.find_all('a', href=lambda href: href and 'TechnicalNotices' in href)

# # Verify that at least one link was found
# if not download_link_tags:
#     raise ValueError("No download links found on the page")

# # Base URL for constructing the full download links
# correct_base_url = "https://backend.taiwanindex.com.tw/api/downloadFile/TechnicalNotices/"

# # Ensure we do not exceed the number of found numbers
# num_links = min(len(download_link_tags), len(target_numbers))

# # Iterate over each found link and download the file
# for i in range(num_links):
#     tag = download_link_tags[i]
#     # Extract the href attribute
#     download_link = tag['href']
    
#     # Construct the full URL
#     file_id = download_link.split('/')[-2]  # Extract the file ID from the URL
#     full_download_url = correct_base_url + file_id + "/tw"
    
#     # Debugging: Print the final download URL
#     print(f"Final download URL for file {i+1}:", full_download_url)
    
#     # Send a GET request to the download link
#     file_response = requests.get(full_download_url)
#     file_response.raise_for_status()  # Check if the request was successful
    
#     # Combine the corresponding found number to the name of the download file
#     file_name = f"{target_numbers[i]}調整成份股.pdf"
    
#     # Save the content to a file
#     with open(file_name, "wb") as file:
#         file.write(file_response.content)
    
#     print(f"Download successful: {file_name}")

# print("All downloads completed.")

### `Step3.Organize text in PDF files.`
> Some of the pdf_directory need to be adjusted because the actions are performed locally on the computer.
>
> Because the format of Taiwanindex is the same, it can be used directly.
>
> Please note that if the format of the pdf file changes in the future, you will need to rewrite the code to organize the file text.

In [1]:
import os
import pdfplumber
import re
import pandas as pd
from datetime import datetime

In [2]:
# Path to the directory containing your PDF files
pdf_directory = 'C:/Python/PGIM_Intern/Web_Crawler/新增資料夾/20241119'

# Regex patterns for extracting desired information
date_pattern = r'(\d{4}年\d{1,2}月\s\d{1,2}日)'
include_pattern = r'成分股納入\(\d+\)\s*：(.*?)成分股刪除\(\d+\)\s*：(.*)'
ticker_pattern = r'(\d{4})\s+(.*?)(、|，|。|$|\n)'
index_name_pattern = r'「(.*?)」成分股'
filename_pattern = r'(\d+)調整成份股\.pdf'

# Lists to store extracted data
data = []


def convert_date(date_str):
    return datetime.strptime(date_str, '%Y年%m月 %d日')

# Iterate through each file in the directory
for filename in os.listdir(pdf_directory):
    if filename.endswith('.pdf'):  # Ensure we're only opening PDF files
        pdf_path = os.path.join(pdf_directory, filename)
        
        # Extract ticker from the filename
        file_ticker_match = re.search(filename_pattern, filename)
        file_ticker = file_ticker_match.group(1) if file_ticker_match else None
        

        pdf = pdfplumber.open(pdf_path)   # Open the PDF file
        for page in pdf.pages:
            text = page.extract_text()    # Extract text from each page

            # Merge all lines into a single line to handle cross-line entries
            text = text.replace('\n', '')  # Remove all newlines to handle multiline content

            # Search for the desired information
            date_match = re.search(date_pattern, text)
            include_match = re.search(include_pattern, text, re.DOTALL)
            index_name_match = re.search(index_name_pattern, text)
            
            if date_match and include_match and index_name_match:
                date_str = date_match.group(0)
                date = convert_date(date_str)
                includes_text = include_match.group(1).strip()
                deletes_text = include_match.group(2).strip()
                index_name = index_name_match.group(1).strip()

                # Extract ticker and name from includes and deletes
                includes = re.findall(ticker_pattern, includes_text)
                deletes = re.findall(ticker_pattern, deletes_text)
                
                # Append data to list
                for ticker, name, _ in includes:
                    cleaned_name = re.sub(r'\*.*$', '', name).strip()  # Remove any trailing annotation
                    data.append([date, ticker, cleaned_name, 1, index_name, file_ticker])
                for ticker, name, _ in deletes:
                    cleaned_name = re.sub(r'\*.*$', '', name).strip()  # Remove any trailing annotation
                    data.append([date, ticker, cleaned_name, -1, index_name, file_ticker])
        
        pdf.close()  # Close the PDF file

# Create a DataFrame
df = pd.DataFrame(data, columns=['開始調整日期', '個股代號', '個股簡稱', '調入/出', '追蹤指數全名', 'ETF代號'])

# Display the DataFrame
df


開始調整日期  個股代號 個股簡稱  調入/出                              追蹤指數全名  ETF代號
0   2024-11-18  2308  台達電     1                  臺灣指數公司特選小資高價 30 指數  00894
1   2024-11-18  2317   鴻海     1                  臺灣指數公司特選小資高價 30 指數  00894
2   2024-11-18  2345   智邦     1                  臺灣指數公司特選小資高價 30 指數  00894
3   2024-11-18  2383  台光電     1                  臺灣指數公司特選小資高價 30 指數  00894
4   2024-11-18  3227   原相     1                  臺灣指數公司特選小資高價 30 指數  00894
..         ...   ...  ...   ...                                 ...    ...
116 2024-11-18  8016   矽創    -1  臺灣指數公司特選臺灣上市上櫃 FactSet 臺灣趨勢動能高股息指數  00944
117 2024-11-18  8070   長華    -1  臺灣指數公司特選臺灣上市上櫃 FactSet 臺灣趨勢動能高股息指數  00944
118 2024-11-18  8112   至上    -1  臺灣指數公司特選臺灣上市上櫃 FactSet 臺灣趨勢動能高股息指數  00944
119 2024-11-18  8150   南茂    -1  臺灣指數公司特選臺灣上市上櫃 FactSet 臺灣趨勢動能高股息指數  00944
120 2024-11-18  9917  中保科    -1  臺灣指數公司特選臺灣上市上櫃 FactSet 臺灣趨勢動能高股息指數  00944

[121 rows x 6 columns]

### `Step4.Download the data from CMoney and perform data processing.`
> The following three types of information are required.
>
>> 1. The net asset information of the ETF requires the previous day's net asset for estimation.(ETF_Net_Asset_Value.xlsx)
>>
>> 2. ETF Constituent Share Weighting for the most recent period.()
>>
>> 3. The last 20 days volume of a stock needs to be counted for average volume.()

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
value = pd.read_excel("ETF_NAV_20241018.xlsx")

value = value.drop(index=[0,1,2])
value = value.reset_index(drop=True)
value = value.transpose()
value.reset_index(drop=True, inplace=True)
value = value.transpose()
value.columns = value.iloc[0]
value = value.drop(index=[0])

value = value.rename(columns={'股票代號': 'ETF代號', '股票名稱': 'ETF簡稱'})

value

In [ ]:
weight = pd.read_excel("ETF_Stocks_Weight_20241018.xlsx")

weight = weight.drop(index=[0,1,2])
weight = weight.reset_index(drop=True)
weight = weight.transpose()
weight.reset_index(drop=True, inplace=True)
weight = weight.transpose()
weight.columns = weight.iloc[0]
weight = weight.drop(index=[0])

weight = weight.rename(columns={'股票代號': 'ETF代號', '股票名稱': 'ETF簡稱','標的代號':'個股代號','權重(%)':'個股佔ETF權重(%)'})

weight['個股代號'] = weight['個股代號'].astype(str)

weight = weight.drop(columns=['日期'],inplace=False)

weight

In [ ]:
stock_20 = pd.read_excel("Average_volume_20days_20241018.xlsx")

stock_20 = stock_20.drop(index=[0,1,2])
stock_20 = stock_20.reset_index(drop=True)
stock_20 = stock_20.transpose()
stock_20.reset_index(drop=True, inplace=True)
stock_20 = stock_20.transpose()
stock_20.columns = stock_20.iloc[0]
stock_20 = stock_20.drop(index=[0])

stock_20 = stock_20.rename(columns={'股票代號': '個股代號', '股票名稱': '個股簡稱'})

stock_20 = stock_20.drop(columns = ['個股簡稱'])

stock_20

In [ ]:
# Step 1: Merge df and value on etf_ticker and 股票代號
merged_df = df.merge(value, left_on='ETF代號', right_on='ETF代號', how='left')
merged_df = merged_df.drop(['追蹤指數全名', 'ETF簡稱'], axis=1)

# Step 2: Merge the resulting DataFrame with weight on etf_ticker and 股票代號
merged_df = merged_df.merge(weight, left_on = ['ETF代號','個股代號'], right_on = ['ETF代號', '個股代號'], how = 'inner')
merged_df = merged_df.drop(['ETF簡稱'], axis=1)

# Step 3: Merge the resulting DataFrame with stock_20 on 個股代號
merged_df = merged_df.merge(stock_20, left_on='個股代號', right_on='個股代號', how='left')


# # Step 4: Calculate the average_day
merged_df['賣壓天數'] = (merged_df['調入/出'] * merged_df['20241018基金淨資產價值(千)'] * merged_df['個股佔ETF權重(%)'] / 100) / merged_df['近20日平均:成交金額(千)']
merged_df

# # Select the desired columns
result_df = merged_df[['開始調整日期', '個股代號', '個股簡稱', '賣壓天數','ETF代號','20241018基金淨資產價值(千)','個股佔ETF權重(%)']]
result_df.columns = ['開始調整日期', '個股代號', '個股簡稱', '賣壓天數(無條件進位)','原ETF代號','ETF調整當日淨資產(千)','個股佔原ETF權重(%)']

result_df = result_df.sort_values(by='賣壓天數(無條件進位)', ascending=True)

result_df

In [ ]:
# This will round up each value to the nearest integer
result_df['賣壓天數(無條件進位)'] = result_df['賣壓天數(無條件進位)'].apply(lambda x: np.ceil(x) if x > 0 else np.floor(x))

In [ ]:
result_df['開始調整日期'] = pd.to_datetime(result_df['開始調整日期'], errors='coerce')

result_df['開始調整日期'] = result_df['開始調整日期'].dt.date
result_df['個股代號'] = pd.to_numeric(result_df['個股代號'], errors='coerce')
result_df



In [ ]:
filename = f"{result_df.loc[0, '開始調整日期'].strftime('%Y-%m-%d')}賣壓天數.xlsx"

result_df.to_excel(filename,index=False)